In [3]:
# sc.stop()
import time
import json
import sys
from pyspark import SparkContext

if __name__ == '__main__':
#     input_json_path = sys.argv[1]
#     output_json_path = sys.argv[2]
    sc = SparkContext.getOrCreate()  
#     try:
#         sc
#     except NameError:
#         sc = SparkContext("local[*]","PySpark Tutorial")
    time_start = time.time()
    input_json_path = 'Books_5.json'
    lines = sc.textFile(input_json_path).coalesce(8).persist()
    n_lines = lines.count()
    print('n_reviews:',n_lines)
    json_read = lines.map(lambda line: json.loads(line))
    reviewer_id_count = json_read.map(lambda line: (line['reviewerID'], 1)).reduceByKey(lambda a, b:a + b).count()
    print('n_reviewer:',reviewer_id_count)
    asin_count = json_read.map(lambda line: (line['asin'], 1)).reduceByKey(lambda a, b:a + b).count()
    print('n_books:',asin_count)
    time_end = time.time()

n_reviews: 8898041
n_reviewer: 603668
n_books: 367982


In [4]:
time_end-time_start

777.650279045105

In [1]:
# sc.stop()
import time
import json
import sys
from pyspark import SparkContext

if __name__ == '__main__':
#     input_json_path = sys.argv[1]
#     output_json_path = sys.argv[2]
    sc = SparkContext.getOrCreate()  
#     try:
#         sc
#     except NameError:
#         sc = SparkContext("local[*]","PySpark Tutorial")
    time_start = time.time()
    input_json_path = 'split_0.json'
    lines = sc.textFile(input_json_path).coalesce(8).persist()
    n_lines = lines.count()
    print('n_reviews:',n_lines)
    json_read = lines.map(lambda line: json.loads(line))
    reviewer_id_count = json_read.map(lambda line: (line['reviewerID'], 1)).reduceByKey(lambda a, b:a + b).count()
    print('n_reviewer:',reviewer_id_count)
    asin_count = json_read.map(lambda line: (line['asin'], 1)).reduceByKey(lambda a, b:a + b).count()
    print('n_books:',asin_count)
    time_end = time.time()

n_reviews: 200000
n_reviewer: 113228
n_books: 5894


In [2]:
time_end-time_start

42.154645681381226

In [10]:
# n_reviewer: 113228
# n_books: 5894
import time
import json
import sys
import numpy as np
input_json_path = 'split_0.json'
n_reviewer = 113228
n_books = 5894
array = np.zeros((1,1))

reviewer_dict = {}
book_dict = {}
# with open(input_json_path,'r',encoding='utf=8') as f:
#     i = 0
#     j = 0
#     for line in f.readlines():
#         line = json.loads(line)
#         if line["reviewerID"] not in reviewer_dict:
#             reviewer_dict[line["reviewerID"]] = i
#             i += 1
#         if line["asin"] not in book_dict:
#             book_dict[line["asin"]] = j
#             j += 1


In [28]:
with open(input_json_path,'r',encoding='utf=8') as f:
    i = 0
    j = 0
    k = 5000
    for line in f.readlines():
        lines = json.loads(line)
        if k == 0:
            break
        if lines["reviewerID"] not in reviewer_dict:
            reviewer_dict[lines["reviewerID"]] = i
            i += 1
            array = np.column_stack((array,np.zeros((array.shape[0],1))))
        if lines["asin"] not in book_dict:
            book_dict[lines["asin"]] = j
            j += 1
            array = np.row_stack((array,np.zeros((1, array.shape[1]))))
        array[book_dict[lines["asin"]],reviewer_dict[lines["reviewerID"]]] = lines["overall"]
        k -= 1
    

In [29]:
array.shape

(43, 4880)

In [30]:
from scipy.sparse.linalg import svds
from numpy.linalg import norm,solve,svd
def get_Ur_hat(A, c, r):
#     Ur_hat
#     A matrix, c draw c columns, r num of top eigenvectors
    if c<1 or r<1:
        c,r = 1, 1
    draw_c_cols = []
    col_prob = norm(A, ord=2, axis=0)**2/norm(A)**2
    col_range = range(0, A.shape[1])
    col_to_choose = np.random.choice(col_range, c, p=col_prob)
    draw_c_cols = A[:, col_to_choose]
    if r == c:
        Ur_u, Ur_sigma, Ur_v = svd(draw_c_cols, full_matrices=False)
    else:
        Ur_u, Ur_sigma, Ur_v = svds(draw_c_cols, k=r)
    return Ur_u

def get_Vr_hat(A, c, r):
#     Vr_hat
#     A matrix, c draw c rows, r num of top eigenvectors
    if c<1 or r<1:
        c,r = 1, 1
    draw_c_rows = []
    row_prob = norm(A, ord=2, axis=1)**2/norm(A)**2
    row_range = range(0, A.shape[0])
    row_to_choose = np.random.choice(row_range, c, p=row_prob)
    draw_c_rows = A[row_to_choose,:]
    if r==c:
        Vr_u, Vr_sigma, Vr_v = svd(draw_c_rows,full_matrices=False)
    else:
        Vr_u, Vr_sigma, Vr_v = svds(draw_c_rows, k=r)
    return Vr_v

Ur_hat = get_Ur_hat(array, int(array.shape[0]/10), int(array.shape[0]/10)-1)
# print(Ur_hat)
Vr_hat = get_Vr_hat(array, int(array.shape[0]/10), int(array.shape[0]/10)-1)

In [31]:
Ur_hat

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.38304317e-16, -3.20694129e-17,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.28656769e-01,  9.73507104e-01,  1.52467617e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00

In [32]:
Vr_hat

array([[8.12993039e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.91740415e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.71585269e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [33]:
a3

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])